# Application of Modern Portfolio Theory to Nifty50
### Using historical data

### Import the required packages

In [46]:
from portfolio_optimizer.data.data_loader import get_nifty50_OHLCV
from portfolio_optimizer.data.data_preprocessor import extract_close_price
from portfolio_optimizer.utils.return_metrics import get_returns, get_mean_returns, get_portfolio_return
from portfolio_optimizer.utils.risk_metrics import get_portfolio_std
from portfolio_optimizer.optimization.mean_variance_optimizer import MeanVariancePortfolioOptimizer

### Load the Data

In [2]:
nifty50_data = get_nifty50_OHLCV()

Using cached data from /home/ashok/Documents/Portfolio_Manager/Portfolio_Optimizer/data/raw/nifty50_OHLCV_5y_1mo.csv


### Display the Data

In [3]:
nifty50_data.head()

Ticker     BAJAJFINSV.NS                                                 \
Price               Open        High         Low       Close     Volume   
Date                                                                      
2020-04-01    455.198085  516.589932  428.045906  509.537354  178844670   
2020-05-01    509.537332  509.537332  397.829133  439.715332  198517650   
2020-06-01    449.208657  644.364862  446.887733  583.487122  371399650   
2020-07-01    580.776834  669.819979  579.244561  619.498657  262393520   
2020-08-01    621.205562  668.821685  604.934240  617.946289  119789060   

Ticker     BHARTIARTL.NS                                                 ...  \
Price               Open        High         Low       Close     Volume  ...   
Date                                                                     ...   
2020-04-01    424.118971  516.605850  397.515165  495.737244  270940205  ...   
2020-05-01    495.737269  589.910953  481.857020  532.654907  544761354  ...   
2020-06-01    538.823853  572.271449  517.617923  539.643188  488577135  ...   
2020-07-01    554.246424  575.259599  523.449588  534.823730  349511506  ...   
2020-08-01    534.486322  544.607366  489.664642  494.580566  536099801  ...   

Ticker         SBIN.NS                                                  \
Price             Open        High         Low       Close      Volume   
Date                                                                     
2020-04-01  182.601392  186.366370  164.717751  179.307037  1133157099   
2020-05-01  179.307039  179.307039  140.668957  151.822708  1246952077   
2020-06-01  154.364079  185.895766  153.752276  167.965057  1835480172   
2020-07-01  168.906286  190.601970  168.106237  180.201218  1537008340   
2020-08-01  180.718920  217.945138  178.412875  199.543808  1455754989   

Ticker      HDFCBANK.NS                                                   
Price              Open         High         Low        Close     Volume  
Date                                                                      
2020-04-01   829.277484   978.218186  777.582661   961.706543  397741571  
2020-05-01   961.706551   961.706551  793.038299   913.563660  383193554  
2020-06-01   935.979143  1039.272818  890.860149  1023.193176  467983388  
2020-07-01  1023.193165  1111.607170  979.226157   991.465942  352780703  
2020-08-01   984.890148  1102.823631  953.258895  1071.192261  292230618  

[5 rows x 250 columns]

### Display the Data Info to know the timeframe of the data and number of entries and other details

In [50]:
nifty50_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2020-04-01 to 2025-03-01
Columns: 250 entries, ('BAJAJFINSV.NS', 'Open') to ('HDFCBANK.NS', 'Volume')
dtypes: float64(200), int64(50)
memory usage: 117.7 KB


### Extract Close Prices from the Data
#### The close prices used are the adjusted ones

In [47]:
nifty50 = extract_close_price(nifty50_data)

### Display the extracted data

In [5]:
nifty50.head()

,BAJAJFINSV,BHARTIARTL,HINDUNILVR,TITAN,TATASTEEL,HDFCLIFE,APOLLOHOSP,TRENT,TATACONSUM,GRASIM,...,DRREDDY,TCS,ULTRACEMCO,NTPC,AXISBANK,NESTLEIND,BAJFINANCE,ONGC,SBIN,HDFCBANK
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-01,509.537354,495.737244,2028.195557,947.615784,12.000605,494.860809,1375.189453,501.906067,338.302612,492.428284,...,705.491821,1886.449097,3449.752930,72.396584,442.969940,1691.255981,2284.737549,60.605518,179.307037,961.706543
2020-05-01,439.715332,532.654907,1901.005981,869.417175,11.875892,516.737183,1338.303345,465.977600,353.023956,574.645874,...,729.678467,1847.024292,3811.438477,74.529228,383.279999,1655.038330,1925.532593,63.108620,151.822708,913.563660
2020-06-01,583.487122,539.643188,2014.335205,927.882935,13.143134,542.218445,1331.300903,611.530396,372.796753,604.392578,...,707.042114,1949.847412,3800.408691,72.967834,404.885864,1620.472046,2790.255371,61.705368,167.965057,1023.193176
2020-07-01,619.498657,534.823730,2055.898926,1019.122986,14.736245,619.353699,1651.588501,554.680786,414.854675,617.412842,...,810.294800,2142.717041,4018.805908,66.265152,429.777405,1564.701904,3204.506592,59.391884,180.201218,991.465942
2020-08-01,617.946289,494.580566,1978.221069,1075.048950,16.614983,567.699768,1612.088745,636.363342,515.939453,658.180237,...,769.274292,2124.790771,3822.476318,73.424820,494.595001,1510.470581,3437.603027,62.160484,199.543808,1071.192261


### Display the company/stock names

In [6]:
nifty50.columns

Index(['BAJAJFINSV', 'BHARTIARTL', 'HINDUNILVR', 'TITAN', 'TATASTEEL',
       'HDFCLIFE', 'APOLLOHOSP', 'TRENT', 'TATACONSUM', 'GRASIM', 'LT',
       'SHRIRAMFIN', 'POWERGRID', 'CIPLA', 'KOTAKBANK', 'SBILIFE',
       'BAJAJ-AUTO', 'M&M', 'INFY', 'ADANIPORTS', 'BPCL', 'HEROMOTOCO',
       'RELIANCE', 'HINDALCO', 'ASIANPAINT', 'EICHERMOT', 'INDUSINDBK',
       'BRITANNIA', 'ADANIENT', 'SUNPHARMA', 'WIPRO', 'ICICIBANK',
       'TATAMOTORS', 'HCLTECH', 'JSWSTEEL', 'ITC', 'MARUTI', 'COALINDIA',
       'BEL', 'TECHM', 'DRREDDY', 'TCS', 'ULTRACEMCO', 'NTPC', 'AXISBANK',
       'NESTLEIND', 'BAJFINANCE', 'ONGC', 'SBIN', 'HDFCBANK'],
      dtype='object')

### Display the high level information of the data
#### to gauge any abnormalities like missing values and undesirable data types, etc

In [7]:
nifty50.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2020-04-01 to 2025-03-01
Data columns (total 50 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BAJAJFINSV  60 non-null     float64
 1   BHARTIARTL  60 non-null     float64
 2   HINDUNILVR  60 non-null     float64
 3   TITAN       60 non-null     float64
 4   TATASTEEL   60 non-null     float64
 5   HDFCLIFE    60 non-null     float64
 6   APOLLOHOSP  60 non-null     float64
 7   TRENT       60 non-null     float64
 8   TATACONSUM  60 non-null     float64
 9   GRASIM      60 non-null     float64
 10  LT          60 non-null     float64
 11  SHRIRAMFIN  60 non-null     float64
 12  POWERGRID   60 non-null     float64
 13  CIPLA       60 non-null     float64
 14  KOTAKBANK   60 non-null     float64
 15  SBILIFE     60 non-null     float64
 16  BAJAJ-AUTO  60 non-null     float64
 17  M&M         60 non-null     float64
 18  INFY        60 non-null     float64
 19  ADANIPORTS 

### Check for the missing values in a systematic manner

In [8]:
# Check for missing values
sum(nifty50.isna().sum())

0

### Calculate and display the Periodic returns (in this case, monthly returns)

In [9]:
returns = get_returns(nifty50)
returns.head()

,BAJAJFINSV,BHARTIARTL,HINDUNILVR,TITAN,TATASTEEL,HDFCLIFE,APOLLOHOSP,TRENT,TATACONSUM,GRASIM,...,DRREDDY,TCS,ULTRACEMCO,NTPC,AXISBANK,NESTLEIND,BAJFINANCE,ONGC,SBIN,HDFCBANK
Date,,,,,,,,,,,,,,,,,,,,,
2020-05-01,-0.137030,0.074470,-0.062711,-0.082521,-0.010392,0.044207,-0.026823,-0.071584,0.043515,0.166964,...,0.034283,-0.020899,0.104844,0.029458,-0.134749,-0.021415,-0.157219,0.041302,-0.153281,-0.050060
2020-06-01,0.326966,0.013120,0.059615,0.067247,0.106707,0.049312,-0.005232,0.312360,0.056010,0.051765,...,-0.031022,0.055670,-0.002894,-0.020950,0.056371,-0.020885,0.449082,-0.022235,0.106324,0.120002
2020-07-01,0.061718,-0.008931,0.020634,0.098331,0.121212,0.142259,0.240582,-0.092963,0.112817,0.021543,...,0.146035,0.098915,0.057467,-0.091858,0.061478,-0.034416,0.148464,-0.037492,0.072849,-0.031008
2020-08-01,-0.002506,-0.075246,-0.037783,0.054877,0.127491,-0.083400,-0.023916,0.147260,0.243663,0.066029,...,-0.050624,-0.008366,-0.048853,0.108046,0.150817,-0.034659,0.072740,0.046616,0.107339,0.080413
2020-09-01,-0.054601,-0.176649,-0.023189,0.099926,-0.106356,-0.026792,0.314154,0.051158,-0.060862,0.109350,...,0.216451,0.104131,0.037322,-0.090034,-0.145143,-0.001937,-0.059981,-0.154973,-0.125472,-0.033383


### Calculate the mean returns of the assets

In [64]:
mean_returns = get_mean_returns(returns)
mean_returns*100 # In percentage

BAJAJFINSV    2.844595
BHARTIARTL    2.229944
HINDUNILVR    0.352545
TITAN         2.296811
TATASTEEL     6.257851
HDFCLIFE      0.706847
APOLLOHOSP    3.082180
TRENT         4.623000
TATACONSUM    2.026785
GRASIM        2.988249
LT            2.541614
SHRIRAMFIN    3.688445
POWERGRID     2.566526
CIPLA         1.822068
KOTAKBANK     0.946822
SBILIFE       1.438440
BAJAJ-AUTO    2.310289
M&M           3.943952
INFY          1.845433
ADANIPORTS    2.894182
BPCL          1.379191
HEROMOTOCO    1.376388
RELIANCE      1.309574
HINDALCO      3.593632
ASIANPAINT    0.810000
EICHERMOT     2.445480
INDUSINDBK    1.463069
BRITANNIA     1.072668
ADANIENT      6.304531
SUNPHARMA     2.547607
WIPRO         2.120115
ICICIBANK     2.389113
TATAMOTORS    4.220722
HCLTECH       2.368343
JSWSTEEL      3.604802
ITC           1.941303
MARUTI        1.584325
COALINDIA     2.841568
BEL           4.855578
TECHM         2.201566
DRREDDY       1.111368
TCS           1.230102
ULTRACEMCO    2.146887
NTPC       

### Display the total number of returned mean return values for sanity check

In [48]:
len(mean_returns)

50

### Get the Variance-Covariance Matrix

In [12]:
# Get the covariance matrix
cov_matrix = returns.cov()
cov_matrix

,BAJAJFINSV,BHARTIARTL,HINDUNILVR,TITAN,TATASTEEL,HDFCLIFE,APOLLOHOSP,TRENT,TATACONSUM,GRASIM,...,DRREDDY,TCS,ULTRACEMCO,NTPC,AXISBANK,NESTLEIND,BAJFINANCE,ONGC,SBIN,HDFCBANK
BAJAJFINSV,0.014960,0.002248,0.002737,0.005538,0.018111,0.002264,0.003979,0.005492,0.004098,0.003036,...,-0.000791,0.000701,0.001929,0.001477,0.004701,0.002168,0.013097,0.001889,0.005069,0.003926
BHARTIARTL,0.002248,0.003904,0.000607,0.000513,-0.000035,0.001232,0.001007,0.000936,0.000887,0.000803,...,-0.000430,0.000535,0.000500,0.001112,0.001448,0.000535,0.001445,0.002209,0.000996,0.000492
HINDUNILVR,0.002737,0.000607,0.004413,0.002619,0.006466,0.002005,0.001064,0.001097,0.002481,0.001061,...,0.000282,0.002038,0.001750,0.001053,0.000935,0.002013,0.002917,-0.000037,0.000202,0.000880
TITAN,0.005538,0.000513,0.002619,0.005801,0.008240,0.001330,0.002729,0.002940,0.002817,0.002321,...,0.001262,0.001593,0.002026,0.001324,0.001851,0.002392,0.005165,0.001167,0.001630,0.001386
TATASTEEL,0.018111,-0.000035,0.006466,0.008240,0.064307,0.001505,0.004893,0.006949,0.007468,0.007469,...,-0.002642,0.000129,0.007122,0.003796,0.007860,0.004107,0.015900,-0.001631,0.008066,0.004372
HDFCLIFE,0.002264,0.001232,0.002005,0.001330,0.001505,0.005029,0.001756,0.001074,0.001509,0.001262,...,0.001120,0.001668,0.001531,0.001228,0.001013,0.001029,0.002082,0.001167,0.001147,0.001102
APOLLOHOSP,0.003979,0.001007,0.001064,0.002729,0.004893,0.001756,0.009882,0.002758,0.002060,0.001400,...,0.002424,0.002177,0.001874,-0.000688,0.000118,0.000585,0.002924,-0.000303,0.000442,0.000300
TRENT,0.005492,0.000936,0.001097,0.002940,0.006949,0.001074,0.002758,0.012026,0.002492,0.001765,...,0.001482,0.000961,0.002090,0.003032,0.003374,0.001082,0.004420,0.001394,0.003845,0.002618
TATACONSUM,0.004098,0.000887,0.002481,0.002817,0.007468,0.001509,0.002060,0.002492,0.005382,0.001989,...,0.000295,0.001676,0.001832,0.002064,0.001979,0.001639,0.004163,0.002112,0.002453,0.001229
GRASIM,0.003036,0.000803,0.001061,0.002321,0.007469,0.001262,0.001400,0.001765,0.001989,0.004965,...,0.000035,0.001225,0.002850,0.002500,0.001599,0.001128,0.002233,0.001823,0.002773,0.001143


### Display Equal Weighted Portfolio's Risk & Return

In [52]:
# Get the portfolio return with equal weights
portfolio_return = get_portfolio_return(mean_returns)
print(f"Portfolio Return: {round(portfolio_return*100,2)}%")
portfolio_std = get_portfolio_std(covariance_matrix=cov_matrix, returns=mean_returns)
print(f"Portfolio Standard Deviation: {round(portfolio_std*100, 2)}%")

Portfolio Return: 2.43%
Portfolio Standard Deviation: 4.59%


## Optimization  based on the specified Objective function and constraints

In [60]:
# possible_constraints = ['long_only', 'required_return']
constraints = ['long_only']
optimizer = MeanVariancePortfolioOptimizer(mean_returns=mean_returns.values, cov_matrix=cov_matrix.values)
# possible_objectives = ['min_volatility', 'max_return', 'sharpe_ratio']
objective = 'sharpe_ratio'
params = {'risk_free_rate': 0.05, 'required_return': 0.06}
weights = optimizer.optimize(objective, constraints, params=params)
weights.round(4)*100 # In percentages

array([ 0.  ,  0.  ,  0.  , -0.  , 19.29,  0.  ,  0.  ,  0.  , -0.  ,
        0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  ,  0.  ,
        0.  , -0.  , -0.  ,  0.  ,  0.  ,  0.  , -0.  , -0.  ,  0.  ,
        0.  , 59.86,  0.  , -0.  ,  0.  ,  0.  ,  0.  , -0.  ,  0.  ,
       -0.  ,  0.  , 20.85,  0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ])

### Diplay the Companies with the corresponding weight in Human Friendly format

In [61]:
nifty50_companies = nifty50.columns
# Filter the companies as per the weights
nifty50_companies[weights.round(4).astype(bool)]

# Create a dictionary with the companies and their weights
weights_dict = dict(zip(nifty50_companies[weights.round(4).astype(bool)], weights.round(4)[weights.round(4).astype(bool)]))
weights_dict

# Print the companies along with the weight in the descending order of the weights
for company, weight in sorted(weights_dict.items(), key=lambda x: x[1], reverse=True):
    print(f"{company}: {round(weight*100,2)}%")

ADANIENT: 59.86%
BEL: 20.85%
TATASTEEL: 19.29%


### Get the Constructed Portfolio's Risk & Return Values

In [62]:
# Get the portfolio return with the portfolio weights
portfolio_return = get_portfolio_return(mean_returns, weights=weights)
print(f"Portfolio Return: {round(portfolio_return*100,2)}%")
portfolio_std = get_portfolio_std(covariance_matrix=cov_matrix, returns=mean_returns, weights=weights)
print(f"Portfolio Standard Deviation: {round(portfolio_std*100, 2)}%")

Portfolio Return: 5.99%
Portfolio Standard Deviation: 13.14%
